In [1]:
#
# Copyright 2020- IBM Inc. All rights reserved
# SPDX-License-Identifier: Apache2.0
#

import warnings
warnings.filterwarnings('ignore')

import threading
import time

import random
import textwrap
import re

import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display, clear_output

import os
import io
from transformers import pipeline, TextClassificationPipeline, RobertaForSequenceClassification, RobertaTokenizerFast
from transformers.utils import logging

In [2]:
%%capture
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter serverextension enable voila --sys-prefix

In [3]:
logging.set_verbosity(40) #error only to suppress warnings from appearing on widget

In [4]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
answerer = pipeline('question-answering', model="deepset/roberta-base-squad2-distilled", tokenizer="deepset/roberta-base-squad2-distilled")
use_model = RobertaForSequenceClassification.from_pretrained("./neurips-roberta/use")
mitigate_model = RobertaForSequenceClassification.from_pretrained("./neurips-roberta/mitigate")
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
classifier1 = TextClassificationPipeline(model=use_model, tokenizer=tokenizer)
classifier2 = TextClassificationPipeline(model=mitigate_model, tokenizer=tokenizer)

In [5]:
# grandson/granddaughter

# input_type = widgets.Select(
#                  options=['Select One', 'File', 'Text'],
#                  value='Select One',
#                  # rows=10,
#                  description='Input Type:',
#                  disabled=False
#              )

text_0 = widgets.HTML(value="<h2>Welcome to the ImpactBot!</h2>")

In [6]:
# name
# uploader = widgets.FileUpload(
#     accept='.txt',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
#     multiple=False  # True to accept multiple files upload else False
# )

# filepath = widgets.Text(placeholder='Write path to input file here')
title_text = widgets.Text(placeholder='Write title here. Keep your title the same throughout the session for data collection.', 
                     layout = Layout(width = '600px'))
input_text = widgets.Textarea(placeholder='Paste impact statement as plain text here. \nWhen ready for feedback, press the "Analyze Impact Statement" button. \n \n \n \n', 
                              layout = Layout(width ='600px'))

In [20]:
button_send = widgets.Button(
                description='Analyze Impact Statement',
                tooltip='Send',
                style={'description_width': 'initial'}, 
                layout = Layout(width = '298px')
            )

button_wrong = widgets.Button(
                description='ImpactBot\'s Classifier is Wrong',
                tooltip='Send',
                style={'description_width': 'initial'}, 
                layout = Layout(width = '298px')
            )

button_done = widgets.Button(
                description='Done',
                tooltip='Send',
                style={'description_width': 'initial'}, 
                layout = Layout(width = '600px')
            )

# reset_button = widgets.Button(
#                 description='RESET',
#                 tooltip='Send',
#                 style={'description_width': 'initial'}, 
#                 layout = Layout(width = '400px')
#             )

output = widgets.Output()

def on_done_clicked(event):
    with output:
        clear_output()
        title_text.value = ""
        input_text.value = ""

def on_wrong_clicked(event):
    with output:
        wrapper = textwrap.TextWrapper(width=75,  
                                       initial_indent='',
                                       subsequent_indent='',
                                       expand_tabs=False,
                                       replace_whitespace=True,
                                       fix_sentence_endings=False,
                                       break_long_words=True,
                                       drop_whitespace=True,
                                       break_on_hyphens=False,
                                       tabsize=8,
                                       max_lines=None,
                                       placeholder=' [...]')
        output_file = "./user-sessions/" + title_text.value + ".txt"
        with open(output_file, "a") as myfile:
            myfile.write("LABEL WRONG \n \n")
        clear_output()
        print(wrapper.fill(text="\033[1mImpactBot:\033[0m Thank you for your feedback. We'll track it to better tune the classifier for future revision."))
#         input_text.value = ""

def on_button_clicked(event):
#     global uploader
    with output:
        clear_output()
        
        wrapper = textwrap.TextWrapper(width=75,  
                                       initial_indent='',
                                       subsequent_indent='',
                                       expand_tabs=False,
                                       replace_whitespace=True,
                                       fix_sentence_endings=False,
                                       break_long_words=True,
                                       drop_whitespace=True,
                                       break_on_hyphens=False,
                                       tabsize=8,
                                       max_lines=None,
                                       placeholder=' [...]')
        
        ### INPUT HANDLING #########################################################################
#         if input_type.value == 'Select One':
#             print("\033[1mImpactBot:\033[0m Sorry you must select an input type first. \n")
#             return
#         elif input_type.value == 'Text':
#             input_statement = input_text.value
#             if input_text.value == 'Paste impact statement as plain text here':
#                 print("\033[1mImpactBot:\033[0m Please paste your impact statement for analysis. \n")
#                 return
#         else: 
#             input_file = list(uploader.value.values())[0]
#             if not input_file:
#                 print("\033[1mImpactBot:\033[0m Please upload your impact statement for analysis. \n")
#                 return
#             content = input_file['content']
#             content = io.StringIO(content.decode('utf-8'))
#             input_statement = content.getvalue()
# #             if not os.path.isfile(file_path):
# #                 print("\033[1mImpactBot:\033[0m Please input an existing filepath. \n")
# #                 return
# #             text_file = open(file_path, "r")
# #             input_statement = text_file.read()
# #             text_file.close()

        if title_text.value == 'Write title here (remember title should be final on submission).' or title_text.value == "":
            print(wrapper.fill(text="\033[1mImpactBot:\033[0m Please first write your title in the box. We need this to help track your impact statement changes."))
            return 
        if input_text.value == 'Paste impact statement as plain text here' or input_text.value == "":
            print(wrapper.fill(text="\033[1mImpactBot:\033[0m Please paste your impact statement for analysis. \n"))
            return
        if len(input_text.value) < 50:
            print(wrapper.fill(text="\033[1mImpactBot:\033[0m Your impact statement should have at least 50 characters to ensure it is thorough."))
            print(wrapper.fill(text="\033[1mImpactBot:\033[0m Consider writing a new impact statement. What are the wanted or expected end results of your research? Who is impacted by it? How?"))
            return
        
        input_statement = input_text.value
#         print(f"Impact Statement: {input_statement} \n\n")
        
        ### SUMMARIZATION PART #####################################################################
        progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, layout = Layout(width = '240px'))
        finished = False
        def work(progress):#method local to this method
            total = 100
            for i in range(total):
                if finished != True:
                    time.sleep(0.2)
                    progress.value = float(i+1)/total
                else:
                    progress.value = 100
                    break

        thread = threading.Thread(target=work, args=(progress,))
        progress_bar_box = widgets.HBox([widgets.Label("ImpactBot - Running Summary, QA, and Classifier Models: "), progress])
        display(progress_bar_box)
        #start the progress bar thread
        thread.start()
        
#         print(f"\033[1mImpactBot:\033[0m Running summarization model. \n")
        
        summarized_output = summarizer(input_statement, 
                                       max_length=min(int(len(input_statement.split(" "))/3), 150), 
                                       min_length=min(int(len(input_statement.split(" "))/10), 25), 
                                       do_sample=False)

    
        
        summary = summarized_output[0].get('summary_text')
        
        print(wrapper.fill(text=f"Summary of Impact Statement: {summary}"))
        print("\n")
        
        ### QA PART ################################################################################
        qa_input = {
            'question': 'What is a potential negative impact or consequence?',
            'context': input_statement
        }
        
        classifier = f"\033[1mImpactBot:\033[0m Asked classifier model to determine if a potential negative impact is mentioned." 
        
        print(wrapper.fill(text=classifier))
              
        qa = "\033[1mImpactBot:\033[0m Asked secondary QA model: \"What is a potential negative impact or consequence?\""
            
        print(wrapper.fill(text=qa))
#         print("\n")
        
        neg_impact_output = answerer(qa_input, max_answer_len=400)
        
        neg_impact = neg_impact_output.get('answer')
#         neg_start = neg_impact_output.get('start')
        possibles1 = [sentence + '.' for sentence in input_statement.split('.') if neg_impact in sentence]
        if possibles1:
            pre1, post1 = possibles1[0].split(neg_impact)
        else:
            pre1, post1 = "", ""
        print("")
#         tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512,'return_tensors':'pt'}
        class1 = classifier1(input_statement, padding=True, truncation=True)
        label1 = class1[0].get('label')
        prob1 = class1[0].get('score')
        finished = True
#         print(label1)
        output_file = "./user-sessions/" + title_text.value + ".txt"
        with open(output_file, "a") as myfile:
            myfile.write("\n")
            myfile.write(input_statement)
            myfile.write("\n")
        if label1 == "LABEL_0":
            with open(output_file, "a") as myfile:
                myfile.write("Use: 0 \n")
            print(wrapper.fill(text=f"\033[1mImpactBot:\033[0m Classifier only found a {(1 - prob1):.0%} probability that your statement contains mention of a potential negative impact."))
#             print("\n")
            print(wrapper.fill(text="\033[1mImpactBot:\033[0m Have you considered what negative societal impacts your project may have?"))
            print("\n")
            print(wrapper.fill(text="\033[1mImpactBot:\033[0m You may have mentioned a negative societal impact. Sadly the classifier isn't always correct."))
            print(wrapper.fill(text=f"\033[1mImpactBot:\033[0m The secondary QA model found a potential answer in the following sentence: \"{pre1[1:]}\033[1m{neg_impact}\033[0m{post1}\""))
#             print(wrapper.fill(text=f"\033[1mImpactBot:\033[0m Secondary model found potential answer st character {neg_start}: \033[1m{neg_impact}\033[0m."))
            print(wrapper.fill(text=f"\033[1mImpactBot:\033[0m You can try revising your impact statement for different results by considering negative societal impacts or press the \"ImpactBot's Classifier is Wrong\" button, to inform us that the classifier was incorrect."))
            return
        else: 
            with open(output_file, "a") as myfile:
                myfile.write("Use: 1 \n")
            print(wrapper.fill(text=f"\033[1mImpactBot:\033[0m Classifier found an {prob1:.0%} probability that your statement contains a potential negative impact"))
            print(wrapper.fill(text=f"\033[1mImpactBot:\033[0m The secondary QA model found a likely answer in the following sentence: \"{pre1[1:]}\033[1m{neg_impact}\033[0m{post1}\""))
            print("\n")
            print(wrapper.fill(text="\033[1mImpactBot:\033[0m Asked the classifier to now determine if statement contains how to address (solve, mitigate, or avoid) a negative impact."))
            qa_input2 = {
                'question': 'What is the way to solve, mitigate, or avoid the negative impact?',
                'context': input_statement
            }
            qa2 = "\033[1mImpactBot:\033[0m Asked secondary QA model: \"What is the way to solve, mitigate, or avoid the negative impact?\""
            print(wrapper.fill(text=qa2))
            mitigate_output = answerer(qa_input2, max_answer_len=400)
            mitigate = mitigate_output.get('answer')
            possibles2 = [sentence + '.' for sentence in input_statement.split('.') if mitigate in sentence]
            if possibles2:
                pre2, post2 = possibles2[0].split(mitigate)
            else:
                pre2, post2 = "", ""
            class2 = classifier2(input_statement, padding=True, truncation=True)
            label2 = class2[0].get('label')
            prob2 = class2[0].get('score')
            if label2 == "LABEL_0":
                with open(output_file, "a") as myfile:
                    myfile.write("Mitigate: 0 \n")
                print("\n")
                print(wrapper.fill(text=f"\033[1mImpactBot:\033[0m Classifier only found a {(1 - prob2):.0%} probability that your statement contains how to address (solve/mitigate/avoid) a potential negative impact."))
                print("\n")
                print(wrapper.fill(text="\033[1mImpactBot:\033[0m Have you considered how to address (solve, mitigate, or avoid) a potential negative societal impact your project may have?"))
                print("\n")
                print(wrapper.fill(text="\033[1mImpactBot:\033[0m You may have mentioned a way to solve, mitigate, or avoid negative impacts. Sadly the classifier isn't always correct."))
                print(wrapper.fill(text=f"\033[1mImpactBot:\033[0m The secondary QA model found a potential answer in the following sentence: \"{pre2[1:]}\033[1m{mitigate}\033[0m{post2}\""))
    #             print(wrapper.fill(text=f"\033[1mImpactBot:\033[0m Secondary model found potential answer st character {neg_start}: \033[1m{neg_impact}\033[0m."))
                print(wrapper.fill(text=f"\033[1mImpactBot:\033[0m You can try revising your impact statement for different results by considering mitigation strategies or press the \"ImpactBot's Classifier is Wrong\" button, to inform us that the classifier was incorrect."))
            else:
                with open(output_file, "a") as myfile:
                    myfile.write("Mitigate: 1 \n")
                qdeep = [
                    "Are there negative impacts that you are actively trying to avoid during your research/development already?",
                    "Which negative impacts of your project are reversible?",
                    "Which negative impacts of your project may be long-lasting or even potentially permanent?",
                    "What are some internal and external metrics for assessing if the project's goals are being met? Are there ways to measure addressing potential negative impacts?",
                    "Where do you NOT intend your work to end up?",
                    "In your work so far, do you see any evidence that actual impacts are deviating from your intent?",
                    "What, if any, are the political and/or legal implications of your project? Are there any regulations to prevent misuse?"
                ]
                print("\n")
                print(wrapper.fill(text=f"\033[1mImpactBot:\033[0m Classifier found an {prob2:.0%} probability that your statement contains how to address (solve/mitigate/avoid) a potential negative impact."))
                print(wrapper.fill(text=f"\033[1mImpactBot:\033[0m The secondary QA model found a likely answer in the following sentence: \"{pre2[1:]}\033[1m{mitigate}\033[0m{post2}\""))
                print(wrapper.fill(text="\033[1mImpactBot:\033[0m Great job. You have mentioned some potentially negative use cases of this technology."))
#                 print("\n")
                print(wrapper.fill(text=f"If you'd like to further consider ways to improve your impact statement, think about the following question. {random.choice(qdeep)}"))
                print(wrapper.fill(text="Otherwise, you may click \"Done\" to finish this session."))
            return
        
        
#       input_str = impact_statement.astype(str).tolist()
#       use_inputs = tokenizer(input_str, padding="max_length", truncation=True)
        
        
        ### HANDLER UPLOADER CLEAR
#         input_type.value = 'Select One'
#         uploader._count = 0
#         uploader.value.clear()

# def on_reset_clicked(event):
#     global uploader
#     uploader = widgets.FileUpload(
#         accept='.txt',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
#         multiple=False  # True to accept multiple files upload else False
#     )
#     with output:
#         clear_output()
#         input_type.value = 'Select One'
#         input_text.value = 'Paste impact statement as plain text here'
#         uploader._count = 0
#         uploader.value.clear()

button_send.on_click(on_button_clicked)
button_wrong.on_click(on_wrong_clicked)
button_done.on_click(on_done_clicked)
# button_reset.on_click(on_reset_clicked)

buttons = widgets.HBox([button_send, button_wrong])
vbox_result = widgets.VBox([buttons, output, button_done])

In [21]:
# stacked right hand side


# text_1 = widgets.HTML(value="<h3>Please first select the input type. We can take your impact statement from a .txt upload or a </h3>")
text_1= widgets.HTML(value="<h4> Please write your title in the textbox below. <br>We use it for data collection so keep it the same throughout. </h4>")
# text_2 = widgets.HTML(value="<em> Please keep the title the same throughout your interaction with the chatbot. We use it to track changes and better tune our models. </em>",
#                       layout = Layout(width= '600px'))
text_3= widgets.HTML(value="<h4> Paste your impact statement in the text box. <br>Press \"Analyze Impact Statement\" when you're ready for feedback. </h3>")

# vbox_text = widgets.VBox([text_0, text_1, input_type, text_2, uploader, input_text, vbox_result])
# vbox_text = widgets.VBox([text_0, text_1, text_2, title_text, text_3, input_text, vbox_result])
vbox_text = widgets.VBox([text_0, text_1, title_text, text_3, input_text, vbox_result])

In [22]:
page = widgets.HBox([vbox_text])
display(page)